# scipy

Stellt verschiedene Untermodule für verschiedene Anwendungszwecke bereit. Eine Auswahl:

- `scipy.special` für spezielle Funktionen wie

- `scipy.linalg` für Kernfunktionen der linearen Algebra (LGS lösen, Determinante usw.)

- `scipy.integrate` für numerische Integration und Lösen von ODEs.

- `scipy.optimize` u.a. für Nullstellen- und Minimierungsprobleme.

- `scipy.interpolate` zum Interpolieren von Funktionen (Splines etc.)

Bemerkung: Der Großteil der Funktionen sind lediglich *Wrapper* für in C oder Fortran geschriebene Routinen. Der Rest ist durch vektorisierte numpy-Funktionen implementiert.

### Lineare Algebra mit `scipy.linalg`

Eigenwerte, Matrixinverse, Lösen linearer Gleichungssyteme und vieles mehr..

**Bemerkung:** Einige Standardfunktionen bzw. -methoden der Linearen Algebra sind (aus Gründen der Rückwärtskompatibilität) sowohl in scipy als auch in numpy vorhanden. **Grundsätzlich sollten jene aus scipy verwendet werden!**

- [Offizielle Dokumentation](https://docs.scipy.org/doc/scipy/reference/linalg.html)
- [Offizielles Tutorial](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html)

In [1]:
import numpy as np
from scipy.linalg import eigvals, inv, solve, det

A = 20*np.eye(5) + np.arange(10,35).reshape(5,5)
print(A, "\n\n", det(A))

[[30. 11. 12. 13. 14.]
 [15. 36. 17. 18. 19.]
 [20. 21. 42. 23. 24.]
 [25. 26. 27. 48. 29.]
 [30. 31. 32. 33. 54.]] 

 18799999.999999996


In [2]:
print(inv(A))

[[ 0.04893617 -0.00255319 -0.00404255 -0.00553191 -0.00702128]
 [-0.00425532  0.04478723 -0.00617021 -0.00712766 -0.00808511]
 [-0.00744681 -0.00787234  0.04170213 -0.0087234  -0.00914894]
 [-0.0106383  -0.01053191 -0.01042553  0.03968085 -0.01021277]
 [-0.01382979 -0.01319149 -0.01255319 -0.01191489  0.0387234 ]]


In [3]:
print(inv(A) @ A)

[[ 1.00000000e+00  1.24032729e-16 -1.11022302e-16  1.47451495e-17
   1.90819582e-17]
 [-6.24500451e-17  1.00000000e+00  0.00000000e+00  4.51028104e-17
  -6.93889390e-18]
 [-9.36750677e-17 -1.30104261e-16  1.00000000e+00 -3.64291930e-17
   3.12250226e-17]
 [-3.81639165e-17  6.41847686e-17 -5.55111512e-17  1.00000000e+00
  -2.42861287e-17]
 [ 4.16333634e-17 -9.02056208e-17  2.22044605e-16  9.02056208e-17
   1.00000000e+00]]


Prüfen, ob $A^{-1}A = E$:

In [4]:
np.all(inv(A) @ A == np.eye(5))

False

Da wir mit Gleitkommazahlen reichnen, macht ein solcher Vergleich natürlich wenig Sinn, besser geeignet ist die Funktion `np.allclose(A,B)`, welche $A$ und $B$ innerhalb einer gewissen Toleranz elementweise auf Gleichheit überprüft

In [5]:
np.allclose(inv(A) @ A, np.eye(5))

True

In [6]:
lambdas = eigvals(A)
print(lambdas)

[132.22761571+0.j  17.77238429+0.j  20.        +0.j  20.        +0.j
  20.        +0.j]


In [7]:
b = np.array([-2, 3 , 4, 8, 10])
sol = solve(A,b)
print(sol)

[-0.23617021 -0.01968085 -0.00319149  0.16329787  0.22978723]


### Optimierung mit `scipy.optimize`

Stellt u.A. Funktionen für

- lineare und nichtlineare (restringierte) Optimierungsprobleme 

- Nullstellenprobleme

- Kleinste-Quadrate- und Funktionsfittingprobleme

bereit.

- [Offizielle Dokumentation](https://docs.scipy.org/doc/scipy/reference/optimize.html)
- [Offizielles Tutorial](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html)

In [8]:
from scipy.optimize import minimize

Als kleines Beispiel wollen wir die Rosenbrockfunktion mit $n=200$ minimieren und wählen als Startpunkt $x_0 = (\frac{11}{10}, \frac{11}{10},\ldots,\frac{11}{10})^{\top}$:

In [9]:
# Möglichkeit 1
def rosenbrock1(x):
    x = np.asarray(x)
    res = 0.0
    for i in range(x.size-1):
        res += 100*(x[i+1]-x[i]**2)**2 + (1-x[i])**2
    return res

# Möglichkeit 2 (vektorisiert)
def rosenbrock2(x):
    x = np.asarray(x)
    return np.sum(100*(x[1:]-x[:-1]**2)**2 + (1-x[:-1])**2)

In [10]:
%%timeit -r 1 -n 1
res1 = minimize(rosenbrock1, x0=1.1*np.ones(200))
print(res1.message)

Optimization terminated successfully.
27.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
%%timeit -r 1 -n 1 
res2 = minimize(rosenbrock2, x0=1.1*np.ones(200))
print(res2.message)

Optimization terminated successfully.
1.38 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Hier sieht man bereits, dass es sich lohnt, Funktionen so effizient wie möglich zu implementieren.

Die Funktion `minimize` gibt ein `OptimizeResult`-Objekt zurück, in welchem alle Informationen in Attributen gespeichert sind, siehe [hier](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult) für eine Übersicht. Den Wert eines Objektattributes enthält man stets durch `objekt.attribut`.
